In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc

In [3]:
num_folds = 5

In [5]:
def mean_predictions(dataset):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        pred = np.zeros_like(test[:, 1:, :, :])
        print(pred.shape)
        
        for appliance in range(test.shape[1]-1):
            for day in range(test.shape[2]):
                pred[:, appliance, day, :] = train[:, appliance+1, day, :].mean(axis=0)
        print(pred[0, 1, 0, 20])
        out.append(pred)
    return np.concatenate(out)


In [24]:
# tensor = np.load('../2015-5appliances.npy')
tensor = {}
tensor[1] = np.load('../2015-5appliances.numpy.npy')
tensor[3] = np.load('../2015-5appliances-subtract-true-agg.npy')
from sklearn.metrics import mean_absolute_error

dataset = 1
err_mean ={}
gt = tensor[dataset][:, 1:, :, :]
pred = mean_predictions(dataset)
# Clamping prediction to aggregate
pred = np.minimum(pred, tensor[dataset][:, 0:1, :,:])
err_mean = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

(14, 5, 112, 24)
98.77932131731951
(14, 5, 112, 24)
100.11327157197175
(14, 5, 112, 24)
94.51203770107693
(13, 5, 112, 24)
95.58969712690873
(13, 5, 112, 24)
102.55484854958274


In [26]:
np.save("../code/baseline/mean-pred-1.npy", pred)

In [25]:
pd.Series(err_mean)

dr        117.771762
dw         28.627193
fridge     28.091365
hvac      316.009448
mw         11.219023
dtype: float64

In [27]:
active_threshold = {'hvac':230, 'fridge':20, 'dr':250, 'dw':55, 'mw':10}

In [32]:
for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
    abs_error = np.abs(pred[:, idx] - tensor[dataset][:, idx+1])
    mask = 1*(tensor[dataset][:, idx+1] > active_threshold[appliance])
    active_error = abs_error[mask == 1].mean()
    inactive_error = abs_error[mask == 0].mean()
    print(appliance, active_error, inactive_error)

hvac 391.28006550483127 102.77426772733615
fridge 27.359565929657403 60.34629937305342
dr 1203.862740561384 61.06450047429224
dw 348.7786252402309 15.037175843772241
mw 48.86999702530569 6.404008685337049
